In [1]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

In [2]:
import os
os.chdir('../')

#### 1.数据读取

In [3]:
CITICIndustryIndexClose = pd.read_excel("./Data/CITICIndustryIndexClose.xlsx",index_col = 0)
CITICIndustryIndexTurnover = pd.read_excel("./Data/CITICIndustryIndexTurnover.xlsx",index_col = 0)
CITICIndustryIndexVolume = pd.read_excel("./Data/CITICIndustryIndexVolume.xlsx",index_col = 0)
CITICIndustryIndexAmount = pd.read_excel("./Data/CITICIndustryIndexAmount.xlsx",index_col = 0)

In [4]:
CITICIndustryIndexClose.index = pd.to_datetime(CITICIndustryIndexClose.index)
CITICIndustryIndexTurnover.index = pd.to_datetime(CITICIndustryIndexTurnover.index)
CITICIndustryIndexVolume.index = pd.to_datetime(CITICIndustryIndexVolume.index)
CITICIndustryIndexAmount.index = pd.to_datetime(CITICIndustryIndexAmount.index)

In [5]:
CITICIndustryIndexClose.ffill(axis = 0,inplace = True)
CITICIndustryIndexAmount.fillna(0,inplace = True)

In [6]:
start_date = pd.to_datetime("2013-12-31")
end_date = pd.to_datetime("2022-04-15")

In [7]:
CITICIndustryIndexClose = CITICIndustryIndexClose.loc[start_date:end_date] # 如果考察2013年之前，那么情况就完全反转
CITICIndustryIndexAmount = CITICIndustryIndexAmount.loc[start_date:end_date] # 如果考察2013年之前，那么情况就完全反转

#### 2.价格拥挤

In [8]:
price_group_1:list = [] # >0%
price_group_2:list = [] # >5%
price_group_3:list = [] 
price_group_4:list = []
price_group_5:list = []
price_group_6:list = []
price_group_7:list = []
price_group_8:list = []
price_group_9:list = []
price_group_10:list = []
price_group_11:list = [] # >50%

In [9]:
# price
for industry in CITICIndustryIndexClose.columns:
    print(f"processing {industry}")
    for date in CITICIndustryIndexClose.index:
        before_close = CITICIndustryIndexClose.loc[:date,industry].iloc[-60:].dropna()
        after_close = CITICIndustryIndexClose.loc[date:,industry].iloc[:60].dropna()
        if len(before_close)<60 or len(after_close)<60:
            continue
        hist_increase = before_close.iloc[-1]/before_close.iloc[0]-1
        corr = pearsonr(before_close.values.tolist(),after_close.values.tolist())
        if pd.isnull(corr):
            continue
        if hist_increase>=0:
            price_group_1.append(corr)
        if hist_increase>=0.05:
            price_group_2.append(corr)
        if hist_increase>=0.10:
            price_group_3.append(corr)
        if hist_increase>=0.15:
            price_group_4.append(corr)
        if hist_increase>=0.20:
            price_group_5.append(corr)
        if hist_increase>=0.25:
            price_group_6.append(corr)
        if hist_increase>=0.30:
            price_group_7.append(corr)
        if hist_increase>=0.35:
            price_group_8.append(corr)
        if hist_increase>=0.40:
            price_group_9.append(corr)
        if hist_increase>=0.45:
            price_group_10.append(corr)
        if hist_increase>=0.50:
            price_group_11.append(corr)

processing CI005001.WI
processing CI005002.WI
processing CI005004.WI
processing CI005008.WI
processing CI005010.WI
processing CI005009.WI
processing CI005007.WI
processing CI005006.WI
processing CI005005.WI
processing CI005003.WI
processing CI005027.WI
processing CI005028.WI
processing CI005026.WI
processing CI005025.WI
processing CI005024.WI
processing CI005023.WI
processing CI005021.WI
processing CI005020.WI
processing CI005019.WI
processing CI005018.WI
processing CI005017.WI
processing CI005016.WI
processing CI005015.WI
processing CI005014.WI
processing CI005013.WI
processing CI005012.WI
processing CI005011.WI
processing CI005259.WI
processing CI005166.WI
processing CI005165.WI


In [10]:
price_groups = [price_group_1,price_group_2,price_group_3,price_group_4,price_group_5,price_group_6,\
                price_group_7,price_group_8,price_group_9,price_group_10,price_group_11]

In [11]:
for i in price_groups:
    corr_avg = np.nanmean(i)
    print(corr_avg) # 确实是有研报中的效应了

0.07691166736498356
0.07387843309496676
0.06677439258913287
0.04533980841256136
0.00861018966736228
-0.032512324756712396
-0.0786154862814475
-0.10900769944378483
-0.1298737554087665
-0.14017004651266868
-0.1529634199061295


#### 3.交易金额拥挤

In [12]:
amount_group_1:list = [] # >0%
amount_group_2:list = [] # >5%
amount_group_3:list = [] 
amount_group_4:list = []
amount_group_5:list = []
amount_group_6:list = []
amount_group_7:list = []
amount_group_8:list = []
amount_group_9:list = []
amount_group_10:list = []
amount_group_11:list = [] # >50%

In [13]:
# amount
for industry in CITICIndustryIndexAmount.columns:
    print(f"processing {industry}")
    for date in CITICIndustryIndexAmount.index:
        before_amount = CITICIndustryIndexAmount.loc[:date,industry].iloc[-60:].dropna()
        before_close = CITICIndustryIndexClose.loc[:date,industry].iloc[-60:].dropna()
        after_close = CITICIndustryIndexClose.loc[date:,industry].iloc[:60].dropna()
        if len(before_close)<60 or len(after_close)<60 or len(before_amount)<60:
            continue
        hist_increase = before_amount.iloc[-1]/before_amount.iloc[0]-1
        corr = pearsonr(before_close.values.tolist(),after_close.values.tolist())
        if pd.isnull(corr):
            continue
        if hist_increase>=0:
            amount_group_1.append(corr)
        if hist_increase>=0.05:
            amount_group_2.append(corr)
        if hist_increase>=0.10:
            amount_group_3.append(corr)
        if hist_increase>=0.15:
            amount_group_4.append(corr)
        if hist_increase>=0.20:
            amount_group_5.append(corr)
        if hist_increase>=0.25:
            amount_group_6.append(corr)
        if hist_increase>=0.30:
            amount_group_7.append(corr)
        if hist_increase>=0.35:
            amount_group_8.append(corr)
        if hist_increase>=0.40:
            amount_group_9.append(corr)
        if hist_increase>=0.45:
            amount_group_10.append(corr)
        if hist_increase>=0.50:
            amount_group_11.append(corr)

processing CI005001.WI
processing CI005002.WI
processing CI005004.WI
processing CI005008.WI
processing CI005010.WI
processing CI005009.WI
processing CI005007.WI
processing CI005006.WI
processing CI005005.WI
processing CI005003.WI
processing CI005027.WI
processing CI005028.WI
processing CI005026.WI
processing CI005025.WI
processing CI005024.WI
processing CI005023.WI
processing CI005021.WI
processing CI005020.WI
processing CI005019.WI
processing CI005018.WI
processing CI005017.WI
processing CI005016.WI
processing CI005015.WI
processing CI005014.WI
processing CI005013.WI
processing CI005012.WI
processing CI005011.WI
processing CI005259.WI
processing CI005166.WI
processing CI005165.WI


In [14]:
amount_groups = [amount_group_1,amount_group_2,amount_group_3,amount_group_4,amount_group_5,amount_group_6,\
                 amount_group_7,amount_group_8,amount_group_9,amount_group_10,amount_group_11]

In [15]:
for i in amount_groups:
    corr_avg = np.nanmean(i)
    print(corr_avg) # 确实有递减，但是为什么都是大于0

0.06969108520129398
0.06792722296535572
0.06718985397190551
0.06584030443946029
0.06419233990013382
0.06256605616720776
0.06051227583532937
0.05832559155272195
0.05586012353723187
0.0532589178011625
0.05110114901020159


In [16]:
amount_group_1:list = [] # >0%
amount_group_2:list = [] # >5%
amount_group_3:list = [] 
amount_group_4:list = []
amount_group_5:list = []
amount_group_6:list = []
amount_group_7:list = []
amount_group_8:list = []
amount_group_9:list = []
amount_group_10:list = []
amount_group_11:list = [] # >50%

In [17]:
# amount
for industry in CITICIndustryIndexAmount.columns:
    print(f"processing {industry}")
    for date in CITICIndustryIndexAmount.index:
        before_amount = CITICIndustryIndexAmount.loc[:date,industry].iloc[-120:-60].dropna()
        after_amount = CITICIndustryIndexAmount.loc[:date,industry].iloc[-60:].dropna()
        before_close = CITICIndustryIndexClose.loc[:date,industry].iloc[-60:].dropna()
        after_close = CITICIndustryIndexClose.loc[date:,industry].iloc[:60].dropna()
        if len(before_amount)<60 or len(after_amount)<60 or len(before_close)<60 or len(after_close)<60:
            continue
        hist_increase = after_amount.sum()/before_amount.sum()-1
        corr = pearsonr(before_close.values.tolist(),after_close.values.tolist())
        if pd.isnull(corr):
            continue
        if hist_increase>=0:
            amount_group_1.append(corr)
        if hist_increase>=0.05:
            amount_group_2.append(corr)
        if hist_increase>=0.10:
            amount_group_3.append(corr)
        if hist_increase>=0.15:
            amount_group_4.append(corr)
        if hist_increase>=0.20:
            amount_group_5.append(corr)
        if hist_increase>=0.25:
            amount_group_6.append(corr)
        if hist_increase>=0.30:
            amount_group_7.append(corr)
        if hist_increase>=0.35:
            amount_group_8.append(corr)
        if hist_increase>=0.40:
            amount_group_9.append(corr)
        if hist_increase>=0.45:
            amount_group_10.append(corr)
        if hist_increase>=0.50:
            amount_group_11.append(corr)

processing CI005001.WI
processing CI005002.WI
processing CI005004.WI
processing CI005008.WI
processing CI005010.WI
processing CI005009.WI
processing CI005007.WI
processing CI005006.WI
processing CI005005.WI
processing CI005003.WI
processing CI005027.WI
processing CI005028.WI
processing CI005026.WI
processing CI005025.WI
processing CI005024.WI
processing CI005023.WI
processing CI005021.WI
processing CI005020.WI
processing CI005019.WI
processing CI005018.WI
processing CI005017.WI
processing CI005016.WI
processing CI005015.WI
processing CI005014.WI
processing CI005013.WI
processing CI005012.WI
processing CI005011.WI
processing CI005259.WI
processing CI005166.WI
processing CI005165.WI


In [18]:
amount_groups = [amount_group_1,amount_group_2,amount_group_3,amount_group_4,amount_group_5,amount_group_6,\
                 amount_group_7,amount_group_8,amount_group_9,amount_group_10,amount_group_11]

In [19]:
for i in amount_groups:
    corr_avg = np.nanmean(i)
    print(corr_avg) # 确实有递减，但是为什么都是大于0

0.08148723128245672
0.08075066565438362
0.07994611062013826
0.07945387273205029
0.07805539017238386
0.07706192857971839
0.07690184806164321
0.07654793191969965
0.07772446016591308
0.07951388730134
0.07890435926354303
